In [1]:
import os
from tqdm.auto import tqdm
from functools import reduce
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import re
import pandas as pd
from bs4 import BeautifulSoup
os.chdir("C:/Users/espen/Documents/SDS/thesis")

tqdm.pandas()

### Load data containing raw html

In [2]:
#Load data
df_html = pd.read_parquet("C:/Users/espen/Documents/SDS/thesis/data/processed/pyarrow/verdicts_in_UfR_only_articles_with_html")
df_html["id_verdict"] = df_html["id"].apply(lambda x: re.sub("\$[0-9]*","",x))


### Define functions to convert HTML to relevant text

In [3]:
def retrieve_html_data(string):
    if type(string) == str:
        soup = BeautifulSoup(string, features="lxml")
        s = soup.find("div", {"class":"maincontent"})
        if s is None: return ""
        s = s.get_text(separator=' ')
        s = re.sub("\n", " ",s)
        s = re.sub(" +"," ",s)
        s = s.split(" Tidsskrifter Ugeskrift for Retsvæsen",1)[0]
    else:
        s = ""
    return s 

### Apply cleaning functions above on each verdict page (there can be multiple pages pr verdict)

In [4]:
df_html["content"] = df_html["html_data"].progress_apply(lambda x: retrieve_html_data(x))

100%|██████████| 68317/68317 [12:09<00:00, 93.69it/s] 


### Merge (potential) multiple pages into one observation pr verdict and clean df

In [5]:
from functools import reduce 
def append_html_body_soup(html_1, html_2=None):
    
    first_page_soup = BeautifulSoup(html_1)
    second_page_soup = BeautifulSoup(html_2)

    [first_page_soup.body.append(element) for element in second_page_soup.body()]
    
    return str(first_page_soup)

In [6]:
for verdict in tqdm(df_html["id_verdict"].unique()):
    if len(pages_df:=df_html.loc[df_html["id_verdict"]==verdict])>1:
        df_html.loc[df_html["id_verdict"]==verdict,"html_concat"]=reduce(append_html_body_soup, list(pages_df["html_data"]))
    else:
        df_html.loc[df_html["id_verdict"]==verdict,"html_concat"]=pages_df["html_data"]

100%|██████████| 63915/63915 [18:40<00:00, 57.02it/s]  


In [7]:
df = df_html.copy()
func = lambda x: " ".join(x)
df = df.loc[:,["content","id_verdict"]].groupby("id_verdict").agg(func).rename(columns={"content":"verdict_text"})
df = pd.merge(df,df_html, left_on="id_verdict",right_on="id_verdict")
df = df.drop_duplicates("id_verdict")
df = df.drop(["id","karnov_pagenation","id_dir_compliant","next_page","prev_page", "url","html_data", "content", "directory"], axis=1)

### Save df as pyarrow table

In [8]:
table = pa.Table.from_pandas(df)
pq.write_table(table,"data/processed/pyarrow/UfR_text.parquet")